# Matrixize Data 

## Purpose:

* Load Data from All Sources 
* Feature Engineering
* Joining Tables
* Output Final Matrices

## Packages and Options

In [1]:
import pandas as pd
import numpy as np 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import mygene

pd.options.display.max_rows=1000000

## Load Data

### Phenotypic Covariates 

In [2]:
# read and set index as subject id number
phenotype = pd.read_csv(
    "/home/jve4pt/resource-files/pulmonary-phenotype/TOPMed_MESA_PFT_CVD_PB_20200722.txt", 
    sep="\t"
).set_index("sidno")

# subset to exam 5 (what we're looking at)
phenotype = phenotype[phenotype["exam"]==5]

# change index to int64 type 
phenotype.index = phenotype.index.astype("int64")

phenotype.head()

/apps/software/standard/core/anaconda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,15,25,103,104,105,106,108,109,110,111) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,age,race,sex,ht_cm,bmi,waist_cm,hip_cm,exam,edu_cat,wt_kg,...,tot_tty_chf,tot_tty_stroke,tot_tty_death,tty_cld_tot_any,cld_event_any,tty_cld_tot_pri,cld_event_pri,race_native,race_others,race_ethnicity
sidno,,,,,,,,,,,,,,,,,,,,,
10001,67,1,1,171.8,23.2061809424555,94.8,97.5,5,5,68.4936,...,14.7214236824093,14.7214236824093,14.7214236824093,14.7214236824093,0,14.7214236824093,0,0,0,1
10003,57,4,0,160.1,48.3117789045806,127.9,148.7,5,4,123.8328,...,11.2306639288159,11.2306639288159,11.2306639288159,11.2306639288159,0,11.2306639288159,0,0,0,4
10004,75,1,0,160.1,24.3151590530747,102,96.8,5,4,62.32464,...,15.1184120465435,15.1184120465435,15.1184120465435,15.1184120465435,0,15.1184120465435,0,0,0,1
10006,77,4,1,158.3,36.4742412772473,117.8,114.7,5,2,91.4004,...,9.94661190965092,12.8843258042437,14.3189596167009,12.8843258042437,1,14.3189596167009,0,0,0,4
10007,55,2,1,169.7,28.5093907269545,96.2,104,5,3,82.1016,...,15.0910335386721,15.0910335386721,15.0910335386721,15.0910335386721,0,15.0910335386721,0,0,0,2


### Expression Matrices

In [3]:
expression_cell_type_and_timepoints = [
    "Monocyte_Exam_5.expression.bed.gz",
    "PBMC_Exam_5.expression.bed.gz",
    "T_cell_Exam_5.expression.bed.gz"
]

In [4]:

expression_matrices = []

#  for each matrix 
for matrix in expression_cell_type_and_timepoints: 
    
    # load and drop the first 3 columns (genomic coordinates)
    # transpose so genes are features and ID's are index
    tmp_df = pd.read_csv(

        "/home/jve4pt/resource-files/expression/{}".format(matrix), 
        compression='gzip', 
        sep="\t"

    ).iloc[:,3:].T
    
    tmp_df.columns = tmp_df.iloc[0]
    tmp_df = tmp_df[1:]
    
    tmp_df.name = matrix

    
    expression_matrices.append(tmp_df)

In [5]:
# make sure they all loaded correctly 
for matrix in expression_matrices: 
    matrix.head()

gene_id,ENSG00000227232.5,ENSG00000268903.1,ENSG00000269981.1,ENSG00000241860.6,ENSG00000279457.4,ENSG00000228463.9,ENSG00000225972.1,ENSG00000225630.1,ENSG00000237973.1,ENSG00000229344.1,...,ENSG00000155959.10,ENSG00000155961.4,ENSG00000155962.12,ENSG00000274791.1,ENSG00000277150.1,ENSG00000224533.4,ENSG00000185973.10,ENSG00000168939.11,ENSG00000124333.15,ENSG00000182484.15
NWD105109,1.21407,2.38988,2.2823,1.24404,1.4336,0.719375,0.613833,0.0704692,0.863904,1.51721,...,-1.19949,0.756352,0.784777,1.4142,0.90556,0.212829,-1.90964,1.58768,-0.498719,1.11688
NWD106213,0.0916624,-0.0422591,-0.863904,-1.37691,0.307547,0.648203,-0.555355,0.948851,-1.14355,-0.220037,...,-0.35966,-1.22892,-0.397483,0.329771,0.784777,-0.00704115,0.248987,-0.285474,-0.0493076,-0.198445
NWD106624,0.784777,0.0422591,0.571862,-1.75856,1.27518,1.56332,0.352159,-1.86774,-0.428154,0.63954,...,-1.4142,-2.06076,-1.39531,0.547159,0.701252,-1.25171,-0.329771,-0.90556,-1.79262,0.155529
NWD115102,-0.701252,0.112897,0.322345,1.05345,-0.344677,-0.490761,0.205632,-0.405115,-0.588525,0.329771,...,-0.285474,-0.605354,-0.833621,0.0563585,-0.0281681,-1.25171,0.804084,-0.0987355,-0.241731,1.02923
NWD118997,0.227256,-0.506708,0.648203,1.90964,1.04126,-0.765758,-0.435883,1.49532,0.630925,0.490761,...,0.141291,-0.0775296,0.993996,-0.965549,-1.86774,0.227256,0.141291,0.737737,-0.119986,1.58768


gene_id,ENSG00000223972.5,ENSG00000227232.5,ENSG00000238009.6,ENSG00000268903.1,ENSG00000269981.1,ENSG00000241860.6,ENSG00000279928.2,ENSG00000279457.4,ENSG00000228463.9,ENSG00000236679.2,...,ENSG00000155959.10,ENSG00000155961.4,ENSG00000155962.12,ENSG00000274791.1,ENSG00000277150.1,ENSG00000185973.10,ENSG00000168939.11,ENSG00000124333.15,ENSG00000124334.17,ENSG00000182484.15
NWD101761,-0.635879,-2.01475,0.223532,-0.82115,-0.332006,-0.719754,0.182116,-0.61124,-0.289427,-0.292451,...,1.17383,1.73221,-0.20574,-0.943032,2.16933,0.16153,0.742469,2.03731,0.353519,0.190963
NWD103464,-0.325889,-0.132236,0.470287,-1.1072,-1.00113,-0.727284,0.179171,-0.866662,-2.11205,-0.298505,...,1.87994,0.833391,0.546138,-0.179171,-0.958973,1.48927,-0.936272,0.909653,1.48054,-0.945294
NWD104274,-0.0739622,-1.47193,-0.972834,-0.552878,-0.837499,-0.0159388,0.618241,-1.40663,0.563036,0.6288,...,0.56984,-0.47677,-0.463823,0.0623438,0.982179,-1.36133,0.450953,1.52537,1.3256,0.0275329
NWD105109,1.50707,-0.496342,0.149797,1.77257,1.91501,0.158594,0.854075,1.71936,0.412779,0.460598,...,-0.44135,-0.319783,1.75879,0.632336,0.250356,-1.28485,0.34428,0.332006,0.406476,0.223532
NWD106213,0.289427,-1.25883,-0.546138,0.0507339,-0.480019,1.1072,-0.0826824,-0.431787,-0.138085,0.447748,...,0.0478325,-0.546138,0.516106,0.0768683,1.52537,0.927326,0.542777,0.141011,0.00724465,-0.887952


gene_id,ENSG00000227232.5,ENSG00000268903.1,ENSG00000269981.1,ENSG00000241860.6,ENSG00000279457.4,ENSG00000228463.9,ENSG00000236679.2,ENSG00000237094.11,ENSG00000225972.1,ENSG00000225630.1,...,ENSG00000155959.10,ENSG00000155961.4,ENSG00000155962.12,ENSG00000274791.1,ENSG00000277150.1,ENSG00000185973.10,ENSG00000168939.11,ENSG00000124333.15,ENSG00000124334.17,ENSG00000182484.15
NWD105109,-1.211,-0.453571,-0.114185,-0.22634,-0.334116,0.827924,1.59733,0.571747,0.468932,0.128098,...,0.780219,0.629647,1.46379,0.646576,0.62125,-1.50536,-0.571747,1.24027,0.212194,-0.430727
NWD106213,-2.54209,0.0933571,-0.385692,-1.76756,-0.515705,-0.672324,0.356103,-0.689739,0.22634,1.96351,...,0.356103,-0.445931,-0.191052,0.492184,0.334116,0.515705,-0.283414,0.951013,-0.877592,-1.00676
NWD106624,1.07809,0.547519,0.547519,1.18273,1.83764,0.612896,2.77562,1.25531,0.91894,0.65511,...,-1.06583,-0.84754,-0.348756,2.77562,2.2897,-2.77562,-1.06583,-0.689739,-0.752433,-0.363471
NWD115102,-0.430727,-1.35221,-0.808622,-1.18273,0.212194,-0.818235,0.689739,-1.76756,-0.212194,-1.73554,...,-0.476654,-0.453571,-1.24027,0.940215,0.385692,-0.867488,-0.588087,-1.70522,-0.128098,0.571747
NWD118997,0.305047,0.857471,1.24027,2.39702,0.672324,-0.91894,-0.415623,0.326823,-0.385692,0.929525,...,-0.0587289,-0.770891,0.984095,-0.385692,-1.83764,1.52716,0.254773,0.484404,0.638089,1.64887


### Covariates

In [6]:
covariate_raw_matrices = [
    "Monocyte_Exam_5.30peers.combined_covariates.txt",
    "PBMC_Exam_5.30peers.combined_covariates.txt",
    "T_cell_Exam_5.30peers.combined_covariates.txt"
]

In [7]:
covariate_matrices = []


for matrix in covariate_raw_matrices: 
    
    # transpose matrix so PC and PEER factors are features and IDs are index
    # drop gender column
    tmp_df = pd.read_csv(
        "/home/jve4pt/resource-files/covariates/{}".format(matrix),
        sep="\t"
    ).T
    
    tmp_df.columns = tmp_df.iloc[0]
    tmp_df = tmp_df[1:]
    tmp_df = tmp_df.drop(columns=["gender"])
    
    tmp_df.name = matrix
    
    covariate_matrices.append(tmp_df)
    

In [8]:
for matrix in covariate_matrices: 
    matrix.shape
    matrix.head()

(355, 41)

ID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
NWD105109,0.0020317,-0.00117186,-0.000868541,-0.00488708,-0.0101016,-0.007257,0.000612204,-0.00483478,-0.00440283,0.0053754,...,-0.0473623,-0.0964618,0.0194492,-0.0386283,-0.0511116,0.000535362,-0.157092,-0.018126,0.0617841,0.0184327
NWD106213,0.00230725,-0.0016403,-0.000853227,0.000871942,0.00137827,-0.000467189,-7.7016e-05,-0.000254521,-0.00145103,-0.00128859,...,0.0181565,0.0647936,-0.0601898,0.0893703,0.050578,-0.0916423,-0.0131243,0.0107403,0.0741159,0.0170431
NWD106624,0.00231519,-0.00169917,-0.000766679,0.000957428,0.00180349,-9.87831e-05,-0.000379841,-0.000798478,0.0015593,0.000914253,...,0.120771,0.0159157,0.0260663,-0.0258128,0.0695602,0.112016,0.0280804,-0.0709703,-0.0394133,-0.12385
NWD115102,0.00198011,-0.00112471,-0.000797524,-0.00545739,-0.0113469,-0.00726904,0.000552443,-0.00616614,-0.00309762,0.00629023,...,-0.101179,-0.110819,-0.0553305,0.102912,-0.0539744,0.0173599,0.0528491,0.0982007,-0.263607,0.0197979
NWD118997,0.00234956,-0.00163289,-0.000799909,0.00167629,0.00277434,-0.00116108,0.000865003,-0.00107694,-0.0030526,0.00104639,...,0.101251,0.0913674,-0.0795181,0.0444321,-0.278748,0.132298,0.128763,-0.0192209,0.0604639,0.126685


(864, 41)

ID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
NWD101761,0.00185328,0.00166972,0.00358127,-0.000484116,-0.000444782,0.000371845,-2.79191e-05,-0.0003858,0.0035507,-0.00246822,...,0.0237859,0.0236259,-0.0717219,-0.0276702,-0.0454595,-0.0122109,0.0861,0.0572442,-0.0621523,0.102847
NWD103464,-0.00487593,0.00125428,0.00247772,7.21474e-05,0.000891155,-0.000172125,0.000458454,-0.000118853,-0.00186354,0.00163209,...,0.0483379,-0.0381697,-0.124039,0.0989314,-0.0326679,-0.0121654,0.102362,0.034156,0.00242032,-0.0339366
NWD104274,0.00230095,-0.00152025,-0.000794307,0.0010031,0.0014459,-0.00288716,0.00360456,0.000643652,-0.0141587,-0.00749303,...,-0.0831534,0.0857599,-0.00962115,-0.019636,0.0243953,-0.00418167,-0.0610086,0.0717253,-0.014835,-0.0564203
NWD105109,0.0020317,-0.00117186,-0.000868541,-0.00488708,-0.0101016,-0.007257,0.000612204,-0.00483478,-0.00440283,0.0053754,...,0.0702075,-0.0155049,-0.0169181,0.0198453,0.00964135,0.0308738,-0.0584951,0.000204498,0.0327878,-0.029936
NWD106213,0.00230725,-0.0016403,-0.000853227,0.000871942,0.00137827,-0.000467189,-7.7016e-05,-0.000254521,-0.00145103,-0.00128859,...,-0.0458264,-0.0153551,-0.00262524,0.0327754,0.0100742,0.0162166,0.00854364,-0.0634473,-0.0570111,0.127771


(362, 41)

ID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
NWD105109,0.0020317,-0.00117186,-0.000868541,-0.00488708,-0.0101016,-0.007257,0.000612204,-0.00483478,-0.00440283,0.0053754,...,0.0358453,0.00139349,-0.0235104,-0.0447376,-0.01606,-0.0132215,0.17574,-0.108359,0.157736,0.048321
NWD106213,0.00230725,-0.0016403,-0.000853227,0.000871942,0.00137827,-0.000467189,-7.7016e-05,-0.000254521,-0.00145103,-0.00128859,...,-0.154227,0.0348658,0.00931623,-0.179022,0.0613906,0.114612,-0.00529452,-0.0218528,-0.0757106,-0.102061
NWD106624,0.00231519,-0.00169917,-0.000766679,0.000957428,0.00180349,-9.87831e-05,-0.000379841,-0.000798478,0.0015593,0.000914253,...,-0.0191633,-0.0259457,0.00840035,0.0395952,-0.0463595,-0.123044,0.0814122,0.173001,0.114332,-0.221354
NWD115102,0.00198011,-0.00112471,-0.000797524,-0.00545739,-0.0113469,-0.00726904,0.000552443,-0.00616614,-0.00309762,0.00629023,...,-0.0314497,-0.0994764,0.0697212,-0.0261592,0.103153,-0.011096,-0.0311864,0.0525439,-0.177128,-0.0155654
NWD118997,0.00234956,-0.00163289,-0.000799909,0.00167629,0.00277434,-0.00116108,0.000865003,-0.00107694,-0.0030526,0.00104639,...,-0.0192248,-0.00725225,-0.130038,-0.0225641,-0.141667,0.0636919,0.151059,0.0894529,0.010222,0.0811145


### Sample Mapping File

In [9]:
sample_id_map = pd.read_csv("/home/jve4pt/resource-files/sample-mapping/freeze9b_sample_annot_2020-08-20.txt", sep="\t")

# subset to MESA participants and set subject_id as index
sample_id_map = sample_id_map[sample_id_map["study"]=="MESA"].set_index("subject_id")
sample_id_map.shape

sample_id_map = sample_id_map.drop(columns=["sex"])

# remove indices with NaN (i.e. no subject_id)
sample_id_map = sample_id_map.loc[sample_id_map.index.dropna()]
sample_id_map.shape

sample_id_map.index = sample_id_map.index.astype("int64")

sample_id_map.head()

/apps/software/standard/core/anaconda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(5382, 15)

(5379, 14)

,sample.id,unique_subject_key,consent,topmed_phs,study,topmed_project,topmed_phase,funding,CCDG_ID,seq_center,geno.cntl,TRIO.dups,Xchr.anom,Ychr.anom
subject_id,,,,,,,,,,,,,,
11037,NWD100343,MESA_11037,HMB,phs001416,MESA,MESA,2,TOPMed,NaN,BROAD,False,False,False,False
14679,NWD100368,MESA_14679,HMB,phs001416,MESA,MESA,2,TOPMed,NaN,BROAD,False,False,False,False
24836,NWD100720,MESA_24836,HMB,phs001416,MESA,MESA,2,TOPMed,NaN,BROAD,False,False,False,False
12615,NWD100793,MESA_12615,HMB,phs001416,MESA,MESA,2,TOPMed,NaN,BROAD,False,False,False,False
14237,NWD101098,MESA_14237,HMB,phs001416,MESA,MESA,2,TOPMed,NaN,BROAD,False,False,False,False


## Feature Engineering before Joining 

### Target Variables

In [10]:
target_variables = phenotype[["pre_fev1", "pre_fvc", "pre_fev1fvc","post_fev1", "post_fvc", "post_fev1fvc"]]

target_variables.head()

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc
sidno,,,,,,
10001,2.057,2.949,69.7524584604951,,,
10003,1.774,2.164,81.9778188539741,1.902,2.292,82.9842931937173
10004,,,,,,
10006,,,,,,
10007,3.146,3.893,80.8117133316209,3.13,3.789,82.6075481657429


### Phenotype DataFrame

In [11]:
# subset to relevant covariates 
phenotype = phenotype[["age","sex", "ht_cm", "wt_kg", "smoking_packyears", "smoking_current", "smoking_former", "race_black","race_chinese","race_hispanic","race_native","race_others","race_white"]]


phenotype.head()

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white
sidno,,,,,,,,,,,,,
10001,67,1,171.8,68.4936,6,0,1,0,0,0,0,0,1
10003,57,0,160.1,123.8328,0,0,0,0,0,1,0,0,0
10004,75,0,160.1,62.32464,0,0,0,0,0,0,0,0,1
10006,77,1,158.3,91.4004,5.6,0,1,0,0,1,0,0,0
10007,55,1,169.7,82.1016,15,0,1,0,1,0,0,0,0


In [12]:
phenotype.index.size

# replace blanks with nan
phenotype.replace(' ', np.nan, inplace=True)
# drop subjects with any value columns 
phenotype = phenotype.dropna()

phenotype.index.size

phenotype.head()

4669

4504

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white
sidno,,,,,,,,,,,,,
10001,67,1,171.8,68.4936,6,0,1,0,0,0,0,0,1
10003,57,0,160.1,123.8328,0,0,0,0,0,1,0,0,0
10004,75,0,160.1,62.32464,0,0,0,0,0,0,0,0,1
10006,77,1,158.3,91.4004,5.6,0,1,0,0,1,0,0,0
10007,55,1,169.7,82.1016,15,0,1,0,1,0,0,0,0


In [13]:
# create dict where each key is a column and value is "float" 
# set all columns to float data type 
phenotype = phenotype.astype(dict(zip(phenotype.columns.to_list(),[ "float" for x in range(phenotype.columns.size)])))
phenotype.head()

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white
sidno,,,,,,,,,,,,,
10001,67.0,1.0,171.8,68.49360,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
10003,57.0,0.0,160.1,123.83280,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10004,75.0,0.0,160.1,62.32464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10006,77.0,1.0,158.3,91.40040,5.6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
10007,55.0,1.0,169.7,82.10160,15.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
# get squared features to model non-linearity 
phenotype["age^2"] = phenotype["age"]**2
phenotype["ht_cm^2"] = phenotype["ht_cm"]**2

phenotype.head()  

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2
sidno,,,,,,,,,,,,,,,
10001,67.0,1.0,171.8,68.49360,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4489.0,29515.24
10003,57.0,0.0,160.1,123.83280,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3249.0,25632.01
10004,75.0,0.0,160.1,62.32464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5625.0,25632.01
10006,77.0,1.0,158.3,91.40040,5.6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89
10007,55.0,1.0,169.7,82.10160,15.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3025.0,28798.09


### Expression Matrices

In [15]:
# check that we have alL ENSEMBL IDs
for matrix in expression_matrices:
    for gene in matrix.columns.to_list(): 
        if not gene.startswith("ENSG"): 
            print("missing ENSEMBL ID")

In [16]:
mg = mygene.MyGeneInfo()
id_mapping_dfs = []

# for each expression matrix 
# get the gene symbol names for ENSEMBL IDs
for matrix in expression_matrices: 
    id_mapping_df = mg.querymany(
        matrix.columns.str.split(".").str[0].to_list(), 
        scopes="ensembl.gene", 
        fields=["symbol"], 
        species="human", 
        returnall=True, 
        as_dataframe=True
    )["out"]

    id_mapping_df["symbol"].value_counts(ascending=False).head()
    
    id_mapping_dfs.append(id_mapping_df)
    

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-16549...done.
Finished.
133 input query terms found no hit:
	['ENSG00000271895', 'ENSG00000242349', 'ENSG00000225986', 'ENSG00000182109', 'ENSG00000261737', 'ENS


RGS5         2
BAZ2B        2
RPL23AP7     2
CYB561D2     2
LINC01238    2
Name: symbol, dtype: int64

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19699...done.
Finished.
196 input query terms found no hit:
	['ENSG00000236269', 'ENSG00000271895', 'ENSG00000242349', 'ENSG00000225986', 'ENSG00000271840', 'ENS


Y_RNA        3
RGS5         2
LINC01238    2
TRIM74       2
BAZ2B        2
Name: symbol, dtype: int64

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19167...done.
Finished.
183 input query terms found no hit:
	['ENSG00000271895', 'ENSG00000242349', 'ENSG00000225986', 'ENSG00000271840', 'ENSG00000116883', 'ENS


Metazoa_SRP    2
BAZ2B          2
RPL23AP7       2
RGS5           2
LINC01238      2
Name: symbol, dtype: int64

In [17]:
for i in range(len(expression_matrices)): 
    
    id_mapping_df = id_mapping_dfs[i]
    matrix = expression_matrices[i]
    
    # make sure each ENSEMBL ID only has one result 
    for ensembl_id in matrix.columns.str.split(".").str[0].to_list(): 
        assert id_mapping_df[id_mapping_df.index==ensembl_id].index.size==1

    index_list = id_mapping_df.index.to_list()
    matrix_genes = matrix.columns.str.split(".").str[0].to_list()

    # make sure the order is preserved between converted symbol df and original input 
    for i in range(len(id_mapping_df.index.to_list())): 
        assert index_list[i] == matrix_genes[i]

In [18]:
# if gene symbol available, replace ENSEMBL ID with that 
# else, keep ENSEMBL ID
for i in range(len(expression_matrices)): 
    new_feature_names = []
    
    id_mapping_df = id_mapping_dfs[i]
    original_names = expression_matrices[i].columns.str.split(".").str[0].to_list()
    
    for j in range(len(original_names)):
        # get gene symbol name as string
        gene_name = str(id_mapping_df[id_mapping_df.index==original_names[j]]["symbol"].to_list()[0])
        
        # if no gene symbol found, keep the ENSEMBL ID
        if gene_name =="nan": 
            new_feature_names.append(original_names[j])
        # else we add the gene symbol
        else: 
            new_feature_names.append(gene_name)
        
    
    expression_matrices[i].columns = new_feature_names
    
    

In [19]:
expression_matrices[0].head()

,WASH7P,ENSG00000268903,ENSG00000269981,ENSG00000241860,WASH9P,RPL23AP21,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,...,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE-AS1,TMLHE,SPRY3,VAMP7,WASH6P
NWD105109,1.21407,2.38988,2.2823,1.24404,1.4336,0.719375,0.613833,0.0704692,0.863904,1.51721,...,-1.19949,0.756352,0.784777,1.4142,0.90556,0.212829,-1.90964,1.58768,-0.498719,1.11688
NWD106213,0.0916624,-0.0422591,-0.863904,-1.37691,0.307547,0.648203,-0.555355,0.948851,-1.14355,-0.220037,...,-0.35966,-1.22892,-0.397483,0.329771,0.784777,-0.00704115,0.248987,-0.285474,-0.0493076,-0.198445
NWD106624,0.784777,0.0422591,0.571862,-1.75856,1.27518,1.56332,0.352159,-1.86774,-0.428154,0.63954,...,-1.4142,-2.06076,-1.39531,0.547159,0.701252,-1.25171,-0.329771,-0.90556,-1.79262,0.155529
NWD115102,-0.701252,0.112897,0.322345,1.05345,-0.344677,-0.490761,0.205632,-0.405115,-0.588525,0.329771,...,-0.285474,-0.605354,-0.833621,0.0563585,-0.0281681,-1.25171,0.804084,-0.0987355,-0.241731,1.02923
NWD118997,0.227256,-0.506708,0.648203,1.90964,1.04126,-0.765758,-0.435883,1.49532,0.630925,0.490761,...,0.141291,-0.0775296,0.993996,-0.965549,-1.86774,0.227256,0.141291,0.737737,-0.119986,1.58768


## Joining Tables

### Check that All Necessary Features are Available for Samples

#### Expression Matrices and Covariate Matrices

In [20]:
for i in range(len(expression_matrices)): 
    set(expression_matrices[i].index.to_list()) - set(covariate_matrices[i].index.to_list())

set()

set()

set()

#### Expression Matrix and Phenotype

In [21]:
phenotype.index.size

# join phenotype table with sample id conversion table
# set sample ID as new index
phenotype = phenotype.join(sample_id_map[["sample.id"]],how="inner").set_index("sample.id")

phenotype.index.size

phenotype.head()

4504

3918

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2
sample.id,,,,,,,,,,,,,,,
NWD578428,67.0,1.0,171.8,68.49360,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4489.0,29515.24
NWD254224,57.0,0.0,160.1,123.83280,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3249.0,25632.01
NWD640007,75.0,0.0,160.1,62.32464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5625.0,25632.01
NWD331470,77.0,1.0,158.3,91.40040,5.6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89
NWD439306,55.0,1.0,169.7,82.10160,15.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3025.0,28798.09


In [22]:
print("These are samples for which we have expression but cannot use phenotypic data: ")

for matrix in expression_matrices: 
    matrix.name
    missing = set(matrix.index.to_list()) - set(phenotype.index.to_list())
    missing 
    
    

These are samples for which we have expression but cannot use phenotypic data: 


'Monocyte_Exam_5.expression.bed.gz'

{'NWD182226',
 'NWD198874',
 'NWD297091',
 'NWD384680',
 'NWD456050',
 'NWD669276',
 'NWD717931',
 'NWD728777',
 'NWD766545'}

'PBMC_Exam_5.expression.bed.gz'

{'NWD182226',
 'NWD198874',
 'NWD220926',
 'NWD273631',
 'NWD297091',
 'NWD456050',
 'NWD484275',
 'NWD598769',
 'NWD613188',
 'NWD669276',
 'NWD717931',
 'NWD766545',
 'NWD857614',
 'NWD947237'}

'T_cell_Exam_5.expression.bed.gz'

{'NWD182226',
 'NWD198874',
 'NWD297091',
 'NWD384680',
 'NWD456050',
 'NWD669276',
 'NWD717931',
 'NWD728777',
 'NWD766545'}

### Begin Joining

#### Target Variables and Phenotype

In [23]:
target_variables.index.size

# inner join and set new index
target_variables = target_variables.join(sample_id_map["sample.id"], how="inner").set_index("sample.id")

target_variables.index.size

target_variables.head()

4669

4036

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc
sample.id,,,,,,
NWD578428,2.057,2.949,69.7524584604951,,,
NWD254224,1.774,2.164,81.9778188539741,1.902,2.292,82.9842931937173
NWD640007,,,,,,
NWD331470,,,,,,
NWD439306,3.146,3.893,80.8117133316209,3.13,3.789,82.6075481657429


In [24]:
# join target variables with phenotype covariates
joined_matrix = target_variables.join(phenotype, how="inner")
joined_matrix.head()

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc,age,sex,ht_cm,wt_kg,...,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2
sample.id,,,,,,,,,,,,,,,,,,,,,
NWD578428,2.057,2.949,69.7524584604951,,,,67.0,1.0,171.8,68.49360,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4489.0,29515.24
NWD254224,1.774,2.164,81.9778188539741,1.902,2.292,82.9842931937173,57.0,0.0,160.1,123.83280,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3249.0,25632.01
NWD640007,,,,,,,75.0,0.0,160.1,62.32464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5625.0,25632.01
NWD331470,,,,,,,77.0,1.0,158.3,91.40040,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89
NWD439306,3.146,3.893,80.8117133316209,3.13,3.789,82.6075481657429,55.0,1.0,169.7,82.10160,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3025.0,28798.09


#### Previous w/ PC/PEER Factors + Expression

In [25]:
final_matrices = []

# join phenotype matrix with PCs/PEER factors and expression matrices 
for i in range(len(covariate_matrices)): 
    
    assert expression_matrices[i].name.split(".")[0] == covariate_matrices[i].name.split(".")[0]
    
    joined_matrix.index.size
    
    tmp_df = joined_matrix.join(covariate_matrices[i], how="inner")
    tmp_df = tmp_df.join(expression_matrices[i], how="inner")
    tmp_df.name = expression_matrices[i].name.split(".")[0]
    
    tmp_df.index.size

    final_matrices.append(tmp_df)
    
    tmp_df.head()
    

3918

346

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc,age,sex,ht_cm,wt_kg,...,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE-AS1,TMLHE,SPRY3,VAMP7,WASH6P
NWD331470,,,,,,,77.0,1.0,158.3,91.40040,...,0.0916624,-0.833621,-0.547159,0.823696,0.467071,-0.141291,-0.804084,0.307547,0.648203,-0.547159
NWD838347,1.759,2.355,74.692144373673,,,,70.0,0.0,163.4,71.66880,...,-1.86774,0.737737,-2.2823,1.51721,0.63954,1.09098,-1.75856,0.337215,-1.39531,1.69599
NWD283680,,,,,,,77.0,0.0,156.8,70.30800,...,-0.241731,0.337215,1.14355,-0.965549,-1.56332,0.613833,-0.112897,1.47412,-0.112897,-0.747012
NWD496227,,,,,,,58.0,1.0,164.6,65.27304,...,-0.155529,0.833621,0.0352127,-0.965549,0.198445,-0.397483,-1.09098,1.0056,-0.205632,0.0845939
NWD420821,2.261,2.969,76.1535870663523,,,,69.0,0.0,154.5,69.62760,...,-0.530875,-0.719375,0.490761,0.959953,0.580173,-1.25171,-2.00518,0.0493076,-0.692276,-0.971176


3918

850

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc,age,sex,ht_cm,wt_kg,...,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE,SPRY3,VAMP7,IL9R,WASH6P
NWD331470,,,,,,,77.0,1.0,158.3,91.4004,...,-0.13516,-0.576671,-1.08084,0.0101426,0.879388,-0.614737,-1.22132,0.338137,0.473526,0.00144892
NWD133576,,,,,,,69.0,1.0,187.6,90.2664,...,-0.301536,-0.757826,-0.845758,0.734855,-0.214627,-0.0391308,-0.56984,0.412779,-0.552878,-1.05017
NWD302101,,,,,,,87.0,0.0,160.5,54.6588,...,0.918453,1.07566,0.719754,-0.943032,-0.597327,1.23363,1.12339,-0.238413,0.202781,-0.556258
NWD838347,1.759,2.355,74.692144373673,,,,70.0,0.0,163.4,71.6688,...,-1.02534,-0.0275329,-0.963573,1.2915,0.247367,-1.43851,-1.52537,-0.614737,-1.5538,0.419098
NWD264729,,,,,,,67.0,0.0,173.4,73.9368,...,-0.789093,-1.19133,-1.54419,0.693714,-0.34428,-0.781205,-0.419098,-0.542777,0.6754,-0.51942


3918

353

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc,age,sex,ht_cm,wt_kg,...,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE,SPRY3,VAMP7,IL9R,WASH6P
NWD331470,,,,,,,77.0,1.0,158.3,91.4004,...,-0.212194,-0.62125,-1.25531,0.523608,0.400613,-0.62125,0.0172641,-0.205136,0.247647,0.555559
NWD838347,1.759,2.355,74.692144373673,,,,70.0,0.0,163.4,71.6688,...,-2.13138,-1.42461,-1.67638,0.725216,0.689739,-0.393142,0.100295,-1.91812,-1.02996,1.46379
NWD283680,,,,,,,77.0,0.0,156.8,70.3080,...,0.408106,-0.799083,1.70522,0.184024,-1.64887,1.59733,-0.128098,0.91894,-1.05372,-0.799083
NWD420821,2.261,2.969,76.1535870663523,,,,69.0,0.0,154.5,69.6276,...,-0.468932,-1.04176,1.15538,0.100295,0.184024,0.135063,0.341427,0.283414,0.523608,-0.867488
NWD638901,1.318,1.909,69.0413829229963,1.372,1.971,69.60933536276,81.0,0.0,159.2,58.2876,...,-1.14203,-1.70522,-0.0310789,-0.995364,-0.363471,1.48426,0.908458,0.539515,1.44392,0.305047


## Split Tables and Output

### Target Variables

In [27]:
for matrix in final_matrices: 
    matrix[["pre_fev1", "pre_fvc", "pre_fev1fvc", "post_fev1", "post_fvc", "post_fev1fvc"]].to_csv(
        "/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/target/{}-target-features.matrix".format(matrix.name), 
        sep="\t"
    )

### Covariates

In [28]:
for matrix in final_matrices:
    matrix.iloc[:,6:62].to_csv(
        "/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/covariates/{}-covariates.matrix".format(matrix.name), 
        sep="\t"
    )

### Expression

In [29]:
for matrix in final_matrices: 
    matrix.iloc[:,62:].to_csv(
        "/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/features/{}-features.matrix".format(matrix.name), 
        sep="\t"
    )